In [1]:
import torch
import torchvision
import torch.nn as nn

import matplotlib.pyplot as plt
import os
import numpy as np


device = 'cuda' if torch.cuda.is_available() else 'cpu'
cnn_model = torchvision.models.resnet34(weights=True)
num_features = cnn_model.fc.in_features
cnn_model.fc = nn.Linear(num_features, 8)
cnn_model.load_state_dict(torch.load('./cnn.pth'))
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(cnn_model.parameters(), lr=0.001, momentum=0.9)



test_dir = './dataset/AI-DATASET/data/test'

transforms_test = torchvision.transforms.Compose([
    torchvision.transforms.Resize((224, 224)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(
        [0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])


c:\Users\kde96\.conda\envs\PyeonHaeng\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\kde96\.conda\envs\PyeonHaeng\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [6]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, files, root, mode='train', transform=None):
        self.files = files
        self.root = root
        self.mode = mode
        self.transform=transform
        
        if 'beverage_can' in files[0]:
            self.label = 0
        elif 'beverage_bottle' in files[0]:
            self.label = 1
        elif 'noodle_bag' in files[0]:
            self.label = 2
        elif 'noodle_cup' in files[0]:
            self.label = 3
        elif 'beverage_milk' in files[0]:
            self.label = 4
        elif 'icecream' in files[0]:
            self.label = 5
        else:
            self.label = 6


    def __len__(self):
        return len(self.files)
    
    def __getitem__(self, index):
        img = plt.Image.open(os.path.join(self.root, self.files[index]))
        
        if self.transform:
            img = self.transform(img)
        if self.mode == 'train':
            return img, np.array([self.label])
        else:
            return img, self.files[index]
        
test_files = [f'{i}.jpg' for i in range(1, 20315+1)]
test_datasets = CustomDataset(files=test_files, root=test_dir, mode='test', transform=transforms_test)
test_dataloader = torch.utils.data.DataLoader(
    test_datasets, batch_size=32, shuffle=False)



In [7]:
imgs, files = iter(test_dataloader).next()